<a href="https://colab.research.google.com/github/penny1xu/Machine-Learning-feature-predict/blob/main/Term_Project_for_CS4375.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install shap

     |████████████████████████████████| 76.3 MB 36.5 MB/s 
     |████████████████████████████████| 564 kB 5.2 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier 
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from catboost import CatBoostClassifier
import math
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import shap
from sklearn.model_selection import cross_val_score

In [ ]:
from pandas.core.arrays import string_
# Train_data
Train_Instance = pd.read_csv("/content/train.mv.txt", sep=" ", header=None, na_values='?')

X = Train_Instance.iloc[: , :-2]
Y = Train_Instance.iloc[: , -2]
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13000 entries, 0 to 12999
Columns: 205 entries, 0 to 204
dtypes: float64(110), int64(95)
memory usage: 20.3 MB


In [ ]:
dataTypeSeries = X.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

Data type of each column of Dataframe :
0        int64
1        int64
2      float64
3      float64
4      float64
        ...   
200      int64
201      int64
202      int64
203      int64
204      int64
Length: 205, dtype: object


In [ ]:
Final = pd.read_csv("/content/final-mv-noclass.txt", sep=" ", header=None, na_values='?')
Final = Final.applymap(str)
Final_X = Final.iloc[: , :-2]
Final_Y = Final.iloc[: , -2]

Final_X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204
0,1995,150,2.0,38.857,-77.0598,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,...,3,2,1,2,3,5,1,1,1,1,1,1,1,9,3,3,3,4,5,3,2,4,3,3,4,5,4,3,6,6,6,2,2,1,1,1,2,1,3,3
1,1998,87,4.0,41.5599,-72.6736,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,nan,nan,nan,nan,nan,...,5,3,2,2,4,6,3,3,2,1,1,1,2,9,4,3,3,4,5,2,2,4,2,2,3,4,4,3,5,5,5,2,3,2,2,2,3,2,3,3
2,1996,18,2.0,39.0131,-76.67699999999999,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,nan,nan,nan,nan,nan,...,3,2,1,2,2,5,1,1,1,1,1,1,1,9,3,3,3,4,5,3,2,4,3,3,4,5,4,3,6,6,6,2,2,1,1,1,2,1,3,3
3,1998,18,3.0,41.9716,-71.638,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,0.0,1.0,0.0,0.0,0.0,...,6,4,2,2,4,7,3,3,2,1,1,1,2,9,6,3,3,4,4,2,2,4,2,2,3,4,4,2,4,5,5,2,3,3,2,3,3,3,3,3
4,1997,31,4.0,38.371,-75.6059,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,3,2,2,2,2,4,1,1,1,1,1,1,1,8,2,3,3,4,5,3,3,4,3,3,4,5,4,4,6,6,6,2,3,2,1,2,2,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2001,68,4.0,40.9832,-72.3134,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,0.0,1.0,1.0,1.0,0.0,...,4,3,2,2,3,6,1,2,1,1,1,1,1,9,3,3,3,4,5,2,2,4,2,2,3,4,4,3,5,6,6,2,3,2,2,2,2,2,3,3
3996,1998,42,4.0,41.1162,-74.0512,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,nan,nan,nan,nan,nan,...,5,3,2,2,3,6,2,2,1,1,1,1,1,9,4,3,3,4,5,2,2,4,2,2,3,4,4,2,5,5,5,2,2,2,1,1,2,1,3,3
3997,2000,123,4.0,38.4544,-76.4947,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.0,1.0,1.0,0.0,0.0,0.0,...,3,2,2,2,2,5,1,1,1,1,1,1,1,9,3,3,3,4,5,3,3,4,3,3,4,5,4,3,6,6,6,2,3,2,1,1,2,1,3,3
3998,1999,85,3.0,40.785,-73.2006,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,3.0,3.0,3.0,3.0,1.0,3.0,1.0,2.0,1.0,nan,nan,nan,nan,nan,...,4,3,2,2,3,6,1,2,1,1,1,1,1,9,3,3,3,4,5,2,2,4,2,2,3,4,4,3,5,6,6,2,2,2,2,1,2,2,3,3


In [ ]:
SEED = 42

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9750 entries, 2487 to 555
Columns: 205 entries, 0 to 204
dtypes: float64(110), int64(95)
memory usage: 15.3 MB


In [ ]:
cat_features_names = X.columns[122:] # here we specify names of categorical features
cat_features = [X.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

[122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204]


In [ ]:
all_features = list(range(X.shape[1]))
print(all_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204]


In [ ]:
import sys
from catboost import Pool

cat_features_names = X.columns[122:] 
cat_features = [X.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

pool = Pool(X_valid, y_valid, cat_features=cat_features)


params = {'loss_function':'Logloss',
          'eval_metric':'AUC',
          'verbose': 200,
          'random_seed': SEED,
          'max_depth': 8,
          'early_stopping_rounds': 400,
          'cat_features':cat_features
         }

cbc_9 = CatBoostClassifier(**params)

all_features = list(range(X.shape[1]))
cbc_10 = cbc_9.select_features(
                X_train,
                y=y_train,
                eval_set=pool,
                features_for_select=all_features,
                num_features_to_select=110,
                algorithm="RecursiveByShapValues",
                steps=2,
                shap_calc_type="Approximate",
                train_final_model=True,
                logging_level="Verbose",
                plot=True,
                log_cout=sys.stdout,
                log_cerr=sys.stderr)


[122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204]


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.055615
Step #1 out of 2
0:	test: 0.7226523	best: 0.7226523 (0)	total: 276ms	remaining: 4m 36s
200:	test: 0.8308846	best: 0.8308846 (200)	total: 38.9s	remaining: 2m 34s
400:	test: 0.8399295	best: 0.8399295 (400)	total: 1m 18s	remaining: 1m 56s
600:	test: 0.8433964	best: 0.8433964 (600)	total: 1m 59s	remaining: 1m 19s
800:	test: 0.8441808	best: 0.8443537 (797)	total: 2m 41s	remaining: 40.1s
999:	test: 0.8444654	best: 0.8446866 (994)	total: 3m 23s	remaining: 0us

bestTest = 0.8446865923
bestIteration = 994

Shrink model to first 995 iterations.
Feature #85 eliminated
Feature #133 eliminated
Feature #121 eliminated
Feature #135 eliminated
Feature #185 eliminated
Feature #119 eliminated
Feature #201 eliminated
Feature #92 eliminated
Feature #84 eliminated
Feature #194 eliminated
Feature #138 eliminated
Feature #67 eliminated
Feature #99 eliminated
Feature #199 eliminated
Feature #17 eliminated
Feature #163 eliminated
Feature #168 eliminated
Feature #182 eliminated
Fea

,feature,importance
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
200,NaN,NaN
201,NaN,NaN
202,NaN,NaN
203,NaN,NaN


In [ ]:
pred_class = cbc_9.predict(X_valid)
pred_class[:30]

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1])

In [ ]:
y_valid.head(30)

12751    1
3176     1
3977     1
2101     1
1108     1
11064    0
12715    1
5410     0
10345    1
2517     0
7207     1
3684     0
6032     0
2291     1
9557     1
9637     0
10068    1
3519     1
1860     1
3322     1
7976     0
4339     1
8097     0
9209     0
2001     0
7366     0
6228     1
2327     0
12034    0
11226    1
Name: 205, dtype: int64

In [ ]:
preds_class = cbc_9.predict(X_valid)
preds_class[:30]

array(['0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '1',
       '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1',
       '0', '1', '1', '0'], dtype=object)

In [ ]:
y_valid.head(30)

2360     1
5918     0
6529     0
6796     1
4579     0
12156    1
8439     1
6862     1
6986     0
8386     0
561      0
519      0
5067     1
264      1
538      1
5857     1
6602     1
3544     1
10986    0
3337     1
9731     0
6693     1
9876     0
8526     1
12693    1
2780     1
571      0
12307    0
3068     0
8019     1
Name: 205, dtype: object

In [ ]:
# Here is the place I generated the Prediction file
from numpy import savetxt

FinalPred_class = cbc_9.predict(Final_X)

results = list(map(int, FinalPred_class))

savetxt('Prediction3.txt', results, delimiter=',', fmt="%d")

NameError: ignored